In [11]:
import geopandas

shapes = geopandas.read_file("../data/raw/Louisville_KY_Downtown_Parking_shapefiles/Louisville_KY_Downtown_Parking.shp")

In [12]:
shapes.columns


Index(['OBJECTID', 'TYPE', 'NAME', 'PARKDT_WEB', 'OWNER_WEB', 'ADDRESS', 'ID',
       'FACILITY', 'FAC_NUM', 'ADDRESS_1', 'CONTACT', 'COMPANY', 'OPERATOR',
       'PHONE_AREA', 'CLASS', 'TYPE_FAC', 'LEVELS', 'TOTAL_SPCS', 'PRIVATE',
       'PVT_PATRON', 'PVT_EMPL', 'PVT_FLEET', 'PVT_RESID', 'PVT_ACCESS',
       'MONTHAVAIL', 'PUB_ACCESS', 'PROPOSED', 'DAY_RATE', 'MONTH_RATE',
       'RSVD_RATE', 'SAT_RATE', 'SUN_RATE', 'RATE_STRU1', 'RATE_STRU2',
       'RATE_STRU3', 'RATE_STRU4', 'RATE_STRU5', 'ADDL_RATE1', 'ADDL_RATE2',
       'ADDL_RATE3', 'MEMO', 'NOTES', 'PARC', 'CITYPARK', 'WEB_DIST', 'LDMD',
       'STR_BLOCK', 'TAX_BLOCK', 'TAX_LOT', 'STREETSORT', 'PUB_SPC',
       'SHAPEAREA', 'SHAPELEN', 'geometry'],
      dtype='object')

In [13]:
to_drop = ["CONTACT", "FAC_NUM", "COMPANY", "OPERATOR", "ID", "LDMD", "STR_BLOCK",
            "MEMO", "NOTES", "CITYPARK", "PROPOSED", "PARC","TOTAL_SPCS", "MONTHAVAIL", "PUB_SPC",
            "PHONE_AREA" ]
exclude_strings = ['RATE', 'TAX', 'WEB', "PVT", "SHAPE", 'PUB_ACCESS', 'STREETSORT']
for string in exclude_strings:
    for col in shapes.columns:
        if string in col:
            to_drop.append(col)

redundant = ['ADDRESS_1', "NAME"]
to_drop.extend(redundant)

#to_drop
cleaner = shapes.drop(to_drop, axis=1)
# renaming to lowercase column names
cleaner = cleaner.rename({column:column.lower() for column in cleaner.columns}, axis=1)

In [14]:
cleaner = cleaner[cleaner['type'] == "G"]
cleaner.head()

,objectid,type,address,facility,class,type_fac,levels,private,geometry
2,3,G,401 W MAIN ST,One Riverfront Plaza,PUBLIC,GARAGE,2,0,"POLYGON ((-85.75727 38.25755, -85.75720 38.257..."
4,5,G,101 W WITHERSPOON,Witherspoon Garage,PUBLIC,GARAGE,3,0,"POLYGON ((-85.75261 38.25795, -85.75252 38.258..."
5,6,G,201 N BROOK ST,Humana Waterside Garage,PUBLIC,GARAGE,5,0,"MULTIPOLYGON (((-85.74826 38.25655, -85.74976 ..."
8,9,G,100 E MAIN ST,First & Main Garage,PUBLIC,GARAGE,6,0,"POLYGON ((-85.75094 38.25532, -85.75151 38.255..."
10,11,G,100 S 2ND ST,Courtyard by Marriott Garage,PUBLIC,GARAGE,7,0,"POLYGON ((-85.75372 38.25556, -85.75407 38.255..."


In [15]:
all(cleaner.private == 0) # all garages are public?
all(cleaner['class'].unique() == 'PUBLIC') # apparently
to_drop_from_garages = ['private', 'class']

cleaner[cleaner['type'] != "G"] # No results
cleaner[cleaner['type_fac'] != "GARAGE"] # No results

to_drop_from_garages.extend(('type', 'type_fac'))

cleaner = cleaner.drop(to_drop_from_garages, axis=1)
cleaner = cleaner.rename({'facility':'facility_name'}, axis=1)
garages = cleaner[['facility_name', 'address', 'levels', 'geometry', 'objectid']]


In [16]:
garages.to_csv("../data/cleaner/parking_garages.csv")